In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
import gdown

# Converte i link forniti in link scaricabili e li scarica nella cartella specificata.
def download_google_file(shader_url, output_name):
  id_url = "https://drive.google.com/uc?id=" + shader_url.split("/")[5]
  gdown.download(id_url, output_name)

# Download MIVIA dataset

In [ ]:
# Scarichiamo i video

download_google_file("https://drive.google.com/file/d/1tEz2wVQjPp1MjVHZLa-Z3uyVBnwljgGF/view?usp=sharing", "VIDEOS.zip")
!unzip VIDEOS.zip

In [ ]:
# Scarichiamo le annotazioni
# Cambiamo i nomi delle directory per semplicità
# GT = Ground Truth

# Per i video classificati 0 il file di annotazione è vuoto 
# Per i video classificati 1 il file di annotazione non è vuoto e in una riga csv
# contiene l'indice del frame per cui per la prima volta l'operatore umano ha visto
# il fumo/fuoco

download_google_file("https://drive.google.com/file/d/123AcAQCldRNE6iKpXuCaVtsaR3uHIOeN/view?usp=sharing", "GT.zip")
!unzip GT.zip
!mkdir -p GT/TRAINING_SET
!mv GT_TRAINING_SET_CL0 GT/TRAINING_SET/0
!mv GT_TRAINING_SET_CL1 GT/TRAINING_SET/1

In [ ]:
# Scarichiamo il codice con i file di test
# Il modello deve essere tale da poter essere eseguibile con questo codice di
# test

download_google_file("https://drive.google.com/file/d/1rXMCtpus2i2UDdSBD9RwWAxnT0wrrXOk/view?usp=sharing", "test_code.zip")
!unzip test_code.zip

In [28]:
videos_path = "TRAINING_SET"
frames_path = "FRAMES"

In [29]:
!rm -R FRAMES/TRAINING_SET/

rm: cannot remove 'FRAMES/TRAINING_SET/': No such file or directory


In [ ]:
import cv2, os, argparse, glob, PIL, tqdm

def extract_frames(video):
    # Process the video
    ret = True
    cap = cv2.VideoCapture(video) # Decodifica lo streaming
    f = 0
    while ret:
        ret, img = cap.read() # Chiamando read leggiamo il frame successivo dallo stream
        if ret: # ret è false quando non ci sono più frame da leggere
            f += 1
            # Il tensore img letto viene trasformato tramite la classe PIL e lo salviamo
            PIL.Image.fromarray(img).save(os.path.join(frames_path, video, "{:05d}.jpg".format(f)))
    cap.release()

# For all the videos
file_list = [path for path in glob.glob(os.path.join(videos_path,"**"), recursive=True)
            if os.path.isfile(path)]
print(file_list)
for video in tqdm.tqdm(file_list):
  if os.path.isdir(os.path.join(frames_path, video)):
    continue
  
  os.makedirs(os.path.join(frames_path, video))
  # Versione lenta che utilizza la funzione definita prima
  #extract_frames(video) 
  # Versione veloce che fa uso della libreria ffmpeg
  os.system("ffmpeg -i {} -r 1/1 {}/{}/$Frame{}.jpg".format(video, frames_path, video, "%05d")) 



In [32]:
!rm *.zip
!rm -r TRAINING_SET

rm: cannot remove '*.zip': No such file or directory


In [33]:
!mkdir MIVIA
!mv FRAMES MIVIA
!mv GT MIVIA

# Download custom dataset

In [ ]:
# Scarichiamo i video

download_google_file("https://drive.google.com/file/d/1eTDG_SbHkCo0OeVwRKugQ2vDV2csDx6q/view?usp=share_link", "VIDEOS.zip")
!unzip VIDEOS.zip

In [ ]:
# Scarichiamo le annotazioni
# Cambiamo i nomi delle directory per semplicità
# GT = Ground Truth

# Per i video classificati 0 il file di annotazione è vuoto 
# Per i video classificati 1 il file di annotazione non è vuoto e in una riga csv
# contiene l'indice del frame per cui per la prima volta l'operatore umano ha visto
# il fumo/fuoco

download_google_file("https://drive.google.com/file/d/1UjWkvzzezXNOkncas4Q-kP9X9VU2D0OE/view?usp=share_link", "GT.zip")
!unzip GT.zip
!mkdir -p GT/TRAINING_SET
!mv GT_TRAINING_SET_CL0 GT/TRAINING_SET/0
!mv GT_TRAINING_SET_CL1 GT/TRAINING_SET/1

In [37]:
!rm *.zip

In [38]:
videos_path = "TRAINING_SET"
frames_path = "FRAMES"

In [ ]:
# For all the videos
file_list = [path for path in glob.glob(os.path.join(videos_path,"**"), recursive=True)
            if os.path.isfile(path)]
print(file_list)
for video in tqdm.tqdm(file_list):
  if os.path.isdir(os.path.join(frames_path, video)):
    continue
  
  os.makedirs(os.path.join(frames_path, video))
  # Versione veloce che fa uso della libreria ffmpeg
  os.system("ffmpeg -i {} -r 1/1 {}/{}/$Frame{}.jpg".format(video, frames_path, video, "%05d")) 

In [40]:
!rm -r TRAINING_SET

In [41]:
!mkdir CUSTOM
!mv FRAMES CUSTOM
!mv GT CUSTOM

In [42]:
!mkdir data
!mv MIVIA data
!mv CUSTOM data

## Dataset

In [ ]:
!pip install striprtf

In [47]:
import os
import os.path
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
from typing import List, Union, Tuple, Any
from striprtf.striprtf import rtf_to_text
import albumentations

class VideoRecord(object):
 # Sostanzilamente mantiene in memoria tutte le informazioni di annotazione di
 # un particolare video
    """
    Helper class for class VideoFrameDataset. This class
    represents a video sample's metadata.

    Args:
        root_datapath: the system path to the root folder of the videos.
        row: A list with four or more elements where
             1) The first element is the path to the video sample's frames excluding
             the root_datapath prefix
             2) The  second element is the starting frame id of the video
             3) The third element is the inclusive ending frame id of the video
             4) The fourth element is the label index.
             5) any following elements are labels in the case of multi-label classification
    """
    def __init__(self, row, root_datapath):
        # row = lista di interi che contiene:
        # 0: path del video
        # 1: primo frame del video (e qui dobbiamo decidere da dove partire)
        # 2: ultimo frame
        # 3: label della classe
        # 4: possono esserci altre annotazioni (es. fumo, fuoco)
        self._data = row
        self._path = os.path.join(root_datapath, row[0])

    @property
    def path(self) -> str:
        return self._path

    @property
    def num_frames(self) -> int:
        return self.end_frame - self.start_frame + 1  # +1 because end frame is inclusive

    @property
    def start_frame(self) -> int:
        return int(self._data[1])

    @property
    def end_frame(self) -> int:
        return int(self._data[2])

    @property
    def label(self) -> Union[int, List[int]]:
        # just one label_id
        if len(self._data) == 4:
            return int(self._data[3])
        # sample associated with multiple labels
        else:
            return [int(label_id) for label_id in self._data[3:]]


class VideoFrameDataset(torch.utils.data.Dataset):
    r"""
    A highly efficient and adaptable dataset class for videos.
    Instead of loading every frame of a video,
    loads x RGB frames of a video (sparse temporal sampling) and evenly
    chooses those frames from start to end of the video, returning
    a list of x PIL images or ``FRAMES x CHANNELS x HEIGHT x WIDTH``
    tensors.

    More specifically, the frame range [START_FRAME, END_FRAME] is divided into NUM_SEGMENTS
    segments and FRAMES_PER_SEGMENT consecutive frames are taken from each segment.

    Note:
        A demonstration of using this class can be seen
        in ``demo.py``
        https://github.com/RaivoKoot/Video-Dataset-Loading-Pytorch

    Note:
        This dataset broadly corresponds to the frame sampling technique
        introduced in ``Temporal Segment Networks`` at ECCV2016
        https://arxiv.org/abs/1608.00859.

    Args:
        root_path: The root path in which video folders lie.
                   this is ROOT_DATA from the description above.
        num_segments: The number of segments the video should
                      be divided into to sample frames from.
        frames_per_segment: The number of frames that should
                            be loaded per segment. For each segment's
                            frame-range, a random start index or the
                            center is chosen, from which frames_per_segment
                            consecutive frames are loaded.
        imagefile_template: The image filename template that video frame files
                            have inside of their video folders as described above.
        transform: Transform pipeline that receives a list of numpy images/frames.
        test_mode: If True, frames are taken from the center of each
                   segment, instead of a random location in each segment.

    """
    def __init__(self,
                 root_path: str, # dove sono contenuti i frame estratti         #? Questo non introdurrebbe dipendenza tra i dati???
                 num_segments: int = 1, # divide il video da cui caricare i
                 # frame in un certo numero di sezioni della stessa durata.
                 frames_per_segment: int = 3, # frame estratti dal segmento in maniera (di default) a caso
                 imagefile_template: str='{:05d}.jpg', # pattern nome dei frame
                 transform=None, # pipeline di augmentation preprocessing
                 totensor=True, # lasciarlo sempre a True
                 test_mode: bool = False): # quanto True, i frame vengono presi
                 # sempre nelle stesse posizioni. È quello che vogliamo fare
                 # quando costruiamo dataset per test o validation
        super(VideoFrameDataset, self).__init__()

        self.root_path = root_path
        self.num_segments = num_segments
        self.frames_per_segment = frames_per_segment
        self.imagefile_template = imagefile_template
        self.test_mode = test_mode

        if transform is None:
            self.transform = None
        else:
            additional_targets = {}
            for i in range(self.num_segments * self.frames_per_segment - 1):
                additional_targets["image%d" % i] = "image"
            self.transform = albumentations.Compose([transform],
                                                    additional_targets=additional_targets,
                                                    p=1)
        self.totensor = totensor
        self.totensor_transform = ImglistOrdictToTensor()

        self._parse_annotationfile()
        self._sanity_check_samples()

    def _load_image(self, directory: str, idx: int) -> Image.Image:
        return np.asarray(Image.open(os.path.join(directory, self.imagefile_template.format(idx))).convert('RGB'))

    def _parse_annotationfile(self):
      # Usando l'organizzazione delle cartelle suggerita l'annotazione viene
      # gestita in automatico. Per ogni file video il codice ricava la posizione
      # del rispettivo file rtf e fa il parsing del file per ricavare la GT
      # In particolare quando vede un video di classe 1, segna il primo frame di
      # avvistamento del fuoco e da lì in poi selezionarà i frame randomici,
      # andando ad ignorare un'eventuale fase del video iniziale in cui il fuoco
      # non c'è
      # Fatto questo parse abbiamo costruito la label del video
        self.video_list = []
        for class_name in os.listdir(self.root_path):
            for video_name in os.listdir(os.path.join(self.root_path, class_name)):
                frames_dir = os.path.join(self.root_path, class_name, video_name)
                if os.path.isdir(frames_dir):
                    frame_path = os.path.join(class_name, video_name)
                    end_frame = len(os.listdir(frames_dir))

                    video_ext = frames_dir.split(".")[-1]
                    annotation_path = frames_dir\
                        .replace("\\", "/") \
                        .replace("FRAMES/", "GT/") \
                        .replace(video_ext, "rtf")

                    with open(annotation_path, 'r') as file:
                        text = rtf_to_text(file.read())
                    if len(text):
                        label = 1
                        start_frame = int(text.split(",")[0])
                        if start_frame == 0:
                          start_frame = 1
                    else:
                        label = 0
                        start_frame = 1

                    self.video_list.append(VideoRecord(
                        [frame_path, start_frame, end_frame, label],
                        self.root_path))

    def _sanity_check_samples(self):
      # Controllo delle annotazioni ricavate per ogni video
        for record in self.video_list:
            if record.num_frames <= 0 or record.start_frame == record.end_frame:
                print(f"\nDataset Warning: video {record.path} seems to have zero RGB frames on disk!\n")

            elif record.num_frames < (self.num_segments * self.frames_per_segment):
                print(f"\nDataset Warning: video {record.path} has {record.num_frames} frames "
                      f"but the dataloader is set up to load "
                      f"(num_segments={self.num_segments})*(frames_per_segment={self.frames_per_segment})"
                      f"={self.num_segments * self.frames_per_segment} frames. Dataloader will throw an "
                      f"error when trying to load this video.\n")

    def _get_start_indices(self, record: VideoRecord) -> 'np.ndarray[int]':
        """
        For each segment, choose a start index from where frames
        are to be loaded from.

        Args:
            record: VideoRecord denoting a video sample.
        Returns:
            List of indices of where the frames of each
            segment are to be loaded from.
        """
        # choose start indices that are perfectly evenly spread across the video frames.
        if self.test_mode:
            distance_between_indices = (record.num_frames - self.frames_per_segment + 1) / float(self.num_segments)

            start_indices = np.array([int(distance_between_indices / 2.0 + distance_between_indices * x)
                                      for x in range(self.num_segments)])
        # randomly sample start indices that are approximately evenly spread across the video frames.
        else:
            max_valid_start_index = (record.num_frames - self.frames_per_segment + 1) // self.num_segments

            start_indices = np.multiply(list(range(self.num_segments)), max_valid_start_index) + \
                      np.random.randint(max_valid_start_index, size=self.num_segments)

        return start_indices

    def __getitem__(self, idx: int) -> Union[
        Tuple[List[Image.Image], Union[int, List[int]]],
        Tuple['torch.Tensor[num_frames, channels, height, width]', Union[int, List[int]]],
        Tuple[Any, Union[int, List[int]]],
        ]:
        """
        For video with id idx, loads self.NUM_SEGMENTS * self.FRAMES_PER_SEGMENT
        frames from evenly chosen locations across the video.

        Args:
            idx: Video sample index.
        Returns:
            A tuple of (video, label). Label is either a single
            integer or a list of integers in the case of multiple labels.
            Video is either 1) a list of PIL images if no transform is used
            2) a batch of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH) in the range [0,1]
            if the transform "ImglistToTensor" is used
            3) or anything else if a custom transform is used.
        """
        record: VideoRecord = self.video_list[idx]

        frame_start_indices: 'np.ndarray[int]' = self._get_start_indices(record)

        return self._get(record, frame_start_indices)

    def _get(self, record: VideoRecord, frame_start_indices: 'np.ndarray[int]') -> Union[
        Tuple[List[Image.Image], Union[int, List[int]]],
        Tuple['torch.Tensor[num_frames, channels, height, width]', Union[int, List[int]]],
        Tuple[Any, Union[int, List[int]]],
        ]:
        """
        Loads the frames of a video at the corresponding
        indices.

        Args:
            record: VideoRecord denoting a video sample.
            frame_start_indices: Indices from which to load consecutive frames from.
        Returns:
            A tuple of (video, label). Label is either a single
            integer or a list of integers in the case of multiple labels.
            Video is either 1) a list of PIL images if no transform is used
            2) a batch of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH) in the range [0,1]
            if the transform "ImglistToTensor" is used
            3) or anything else if a custom transform is used.
        """

        frame_start_indices = frame_start_indices + record.start_frame
        images = list()

        # from each start_index, load self.frames_per_segment
        # consecutive frames
        for start_index in frame_start_indices:
            frame_index = int(start_index)

            # load self.frames_per_segment consecutive frames
            for _ in range(self.frames_per_segment):
                image = self._load_image(record.path, frame_index)
                images.append(image)

                if frame_index < record.end_frame:
                    frame_index += 1

        if self.transform is not None:
            transform_input = {"image": images[0]}
            for i, image in enumerate(images[1:]):
                transform_input["image%d" % i] = image
            images = self.transform(**transform_input)

        if self.totensor:
            images = self.totensor_transform(images)
        return images, record.label

    def __len__(self):
        return len(self.video_list)


class ImglistOrdictToTensor(torch.nn.Module):
    """
    Converts a list or a dict of numpy images to a torch.FloatTensor
    of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH).
    Can be used as first transform for ``VideoFrameDataset``.
    """
    @staticmethod
    def forward(img_list_or_dict):
        """
        Converts each numpy image in a list or a dict to
        a torch Tensor and stacks them into a single tensor.

        Args:
            img_list_or_dict: list or dict of numpy images.
        Returns:
            tensor of size ``NUM_IMAGES x CHANNELS x HEIGHT x WIDTH``
        """
        if isinstance(img_list_or_dict, list):
            return torch.stack([transforms.functional.to_tensor(img)
                                for img in img_list_or_dict])
        else:
            return torch.stack([transforms.functional.to_tensor(img_list_or_dict[k])
                                for k in img_list_or_dict.keys()])


In [48]:
import torch.nn as nn

class Model(nn.Module):
    
    def __init__(self, num_classes=1, to_train=0):
        
        super(Model, self).__init__()
        self.num_classes = num_classes
        self.to_train = to_train

    def forward(self, x): 
        pass
    
    def get_trainable_parameters(self):
        return [p for p in self.parameters() if p.requires_grad]
    
    def get_preprocessing(self):
        pass

In [ ]:
!pip install pytorchvideo

In [52]:
import torch
import torch.nn as nn
import torchvision.models as models
import albumentations
from pytorchvideo.models import create_res_basic_head

class X3D_xs(Model):
    
    def __init__(self, num_classes=1, to_train=0):
        
        super().__init__(num_classes=num_classes, to_train=to_train)

        # Preprocessing parameters
        side_size = 182
        crop_size = 182
        mean = [0.45, 0.45, 0.45]
        std = [0.225, 0.225, 0.225]

        self.preprocessing = albumentations.Sequential([
            albumentations.SmallestMaxSize(side_size, always_apply=True),
            albumentations.CenterCrop(crop_size, crop_size, always_apply=True),
            albumentations.Normalize(mean=mean,
                                        std=std,
                                        max_pixel_value=255.,
                                        always_apply=True), 
        ])

        # Model parameters
        self.model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs', pretrained=True)

        # Replace the last layer for finetuning
        self.model.blocks[:-to_train].requires_grad_(False)
        self.model.blocks[-1] = create_res_basic_head(in_features=192, out_features=num_classes, pool_kernel_size=(1, 6, 6))

    def forward(self, x):
        # x è un batch di frame di video: B x T x C x H x W oppure T x C x H x W
        # B = batch size, T = numero di frame, C = numero di canali, H = altezza, W = larghezza
        if len(x.size()) == 4:
            x = x.unsqueeze(0)

        return self.model(x)

In [ ]:
num_classes = 1
to_train = 2

model = X3D_xs(num_classes, to_train)
model

## Augumentations

In [54]:
import cv2

# Preprocessing and augmentation
# applica sempre, ha senso per il preprocessing
preprocessing = albumentations.Sequential([
    albumentations.Resize(height=256, width=256, always_apply=True),
    albumentations.CenterCrop(height=224, width=224, always_apply=True),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225],
                                max_pixel_value=255.,
                                always_apply=True),
])

augmentation = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.2), # Flip orizzontale
    albumentations.Rotate(p=1., limit=30), # Rotazione
    albumentations.Blur(p=0.2), # Sfoca l'immagine
    albumentations.Sharpen(p=0.2),  # Accentua i bordi
], p=.5)

# Dataset Split

In [69]:
!rm -r data/split

In [71]:
import os
import random
import shutil

def split_dataset(source_folder, dest_folder, percentages):
    
    # cartella per le annotazioni + cartella per i frames
    new_folders = ['FRAMES','GT']
    
    for new_folder in new_folders:
        new_path = os.path.join(dest_folder,new_folder)
        # crea le cartelle di destinazione
        for folder in ['TRAINING_SET', 'VALIDATION_SET', 'TEST_SET']:
            for video_type in ['0','1']:
                os.makedirs(os.path.join(new_path, folder, video_type), exist_ok=True)

    original_frames_path = os.path.join(source_folder, new_folders[0], 'TRAINING_SET')
    original_annotations_path = os.path.join(source_folder, new_folders[1], 'TRAINING_SET')
    
        # Ottieni la lista di cartelle VideoX.mp4
    video_folders_0 = [folder_name for folder_name in os.listdir(os.path.join(original_frames_path, '0')) if
                     os.path.isdir(os.path.join(original_frames_path, '0', folder_name))]
    video_folders_1 = [folder_name for folder_name in os.listdir(os.path.join(original_frames_path, '1')) if
                     os.path.isdir(os.path.join(original_frames_path, '1', folder_name))]

    random.shuffle(video_folders_0)  # Mischia l'ordine delle cartelle
    random.shuffle(video_folders_1)  # Mischia l'ordine delle cartelle

    # Calcola il numero di video per ciascuna categoria
    total_videos_0 = len(video_folders_0)
    total_videos_1 = len(video_folders_1)
    train_count_0 = int(total_videos_0 * percentages[0])
    val_count_0 = int(total_videos_0 * percentages[1])
    test_count_0 = total_videos_0 - train_count_0 - val_count_0
    train_count_1 = int(total_videos_1 * percentages[0])
    val_count_1 = int(total_videos_1 * percentages[1])
    test_count_1 = total_videos_1 - train_count_1 - val_count_1

    # Copia i file nelle nuove cartelle per la categoria 0
    for i, video_folder in enumerate(video_folders_0):
        source_video_path = os.path.join(original_frames_path, '0', video_folder)
        source_annot_path = os.path.join(original_annotations_path, '0', video_folder[:-4] + '.rtf')

        if i < train_count_0:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'TRAINING_SET', '0', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'TRAINING_SET', '0', video_folder[:-4] + '.rtf')
        elif i < train_count_0 + val_count_0:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'VALIDATION_SET', '0', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'VALIDATION_SET', '0', video_folder[:-4] + '.rtf')
        else:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'TEST_SET', '0', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'TEST_SET', '0', video_folder[:-4] + '.rtf')

        shutil.copytree(source_video_path, dest_video_path)
        shutil.copyfile(source_annot_path, dest_annot_path)

    # Copia i file nelle nuove cartelle per la categoria 1
    for i, video_folder in enumerate(video_folders_1):
        source_video_path = os.path.join(original_frames_path, '1', video_folder)
        source_annot_path = os.path.join(original_annotations_path, '1', video_folder[:-4] + '.rtf')

        if i < train_count_1:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'TRAINING_SET', '1', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'TRAINING_SET', '1', video_folder[:-4] + '.rtf')
        elif i < train_count_1 + val_count_1:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'VALIDATION_SET', '1', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'VALIDATION_SET', '1', video_folder[:-4] + '.rtf')
        else:
            dest_video_path = os.path.join(dest_folder, 'FRAMES', 'TEST_SET', '1', video_folder)
            dest_annot_path = os.path.join(dest_folder, 'GT', 'TEST_SET', '1', video_folder[:-4] + '.rtf')

        shutil.copytree(source_video_path, dest_video_path)
        shutil.copyfile(source_annot_path, dest_annot_path)

    print("Split completato.")

split_dataset('data/MIVIA', 'data/split', [0.6, 0.2, 0.2])

Split completato.


In [74]:
# Dataset for training
train_dataset = VideoFrameDataset(root_path="data/split/FRAMES/TRAINING_SET/",
                            num_segments=3,
                            frames_per_segment=1,
                            transform=albumentations.Compose([
                                preprocessing,
                                augmentation],
                                p=1.,
                            )
                            )

val_dataset = VideoFrameDataset(root_path="data/split/FRAMES/VALIDATION_SET/",
                            num_segments=3,
                            frames_per_segment=1,
                            transform=albumentations.Compose([
                                preprocessing,
                                augmentation],
                                p=1.,
                            ),
                            test_mode=True
                            )

In [79]:
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook

def start_tensorboard(log_dir):
  writer = SummaryWriter(os.path.join("experiments","runs", log_dir))

  # run tensorboard in background
  ! killall tensorboard
  %load_ext tensorboard
  %tensorboard --logdir ./runs

  notebook.list() # View open TensorBoard instances

  return writer

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [75]:
from torchvision.utils import make_grid
from tqdm import tqdm

def one_epoch(model, lossFunction, output_activation, optimizer, train_loader, val_loader, writer, epoch_num):
  model = model.to(device)

  model.train()

  i_start = epoch_num * len(train_loader)
  for i, (X, y) in tqdm(enumerate(train_loader), desc="epoch {} - train".format(epoch_num)):

    X = X.to(device)
    y = y.to(device).float()

    optimizer.zero_grad()

    o = model(X)
    o = output_activation(o)

    # make shape of o and y the same
    if o.shape != y.shape:
      y = y.reshape(o.shape)

    l = lossFunction(o, y)

    l.backward()
    optimizer.step()

    acc = ((o.detach() > .5) == y.detach()).float().mean()

    # print("- batch loss and accuracy : {:.7f}\t{:.4f}".format(l.detach().item(), acc))
    writer.add_scalar('train/loss', l.detach().item(), i_start+i)
    writer.add_scalar('train/acc', acc, i_start+i)

  model.eval()
  with torch.no_grad():
    val_loss = []
    val_corr_pred = []
    for X, y in tqdm(val_loader, desc="epoch {} - validation".format(epoch_num)):

      X = X.to(device)
      y = y.to(device).float()

      o = model(X)
      o = output_activation(o).squeeze()
      val_loss.append(lossFunction(o, y))
      val_corr_pred.append((o > .5) == y)

    val_loss = torch.stack(val_loss).mean().item()
    val_accuracy = torch.concatenate(val_corr_pred).float().mean().item()

    # print("Validation loss and accuracy : {:.7f}\t{:.4f}".format(val_loss, val_accuracy))
    writer.add_scalar('val/loss', val_loss, i_start+i)
    writer.add_scalar('val/acc', val_accuracy, i_start+i)
  return val_loss, val_accuracy

In [78]:
from torch.nn import BCELoss, CrossEntropyLoss, Sigmoid, Softmax
from datetime import datetime

# learning parameters
lossFunction, output_activation = BCELoss(), Sigmoid()
batch_size = 64
lr = .0001
momentum = .9
lambda_reg = 0

epochs = 200
early_stopping_patience = 10

optimizer = torch.optim.Adam(model.get_trainable_parameters(),
                          lr=lr,
                          weight_decay=lambda_reg)

model_name = "x3d_xs"
# create output directory and logger
name = model_name+"_"+datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = os.path.join("experiments", name)

In [81]:
import os
os.makedirs(experiment_name)
writer = start_tensorboard(name)
!reload_ext tensorboard

/bin/bash: line 1: killall: command not found


Known TensorBoard instances:
  - port 6006: logdir ./runs (started 0:00:00 ago; pid 7205)
/bin/bash: line 1: reload_ext: command not found


In [82]:
from torch.utils.data import DataLoader
dataloader_params = {"batch_size": batch_size, "num_workers": 0, "pin_memory": True}
train_loader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
val_loader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# early stopping and best model saving
early_stopping_counter = early_stopping_patience
min_val_loss = 1e10

# training and validation
val_losses = torch.zeros(epochs)
val_accuracies = torch.zeros(epochs)
for e in range(epochs):
  print("EPOCH {}".format(e))
  val_loss, val_accuracy = one_epoch(model, lossFunction, output_activation, optimizer, train_loader, val_loader, writer, e)

  # store the validation metrics
  val_losses[e] = val_loss
  val_accuracies[e] = val_accuracy
  torch.save(val_losses, os.path.join(experiment_name,'val_losses.pth'))
  torch.save(val_accuracies, os.path.join(experiment_name,'val_accuracies.pth'))

  # save the best model and check the early stopping criteria
  if val_loss < min_val_loss: # save the best model
    min_val_loss = val_loss
    early_stopping_counter = early_stopping_patience # reset early stopping counter
    torch.save(model.state_dict(), os.path.join(experiment_name,'best_model.pth'))
    print("- saved best model: val_loss =", val_loss, "val_accuracy =", val_accuracy)

  if e>0: # early stopping counter update
    if val_losses[e] > val_losses[e-1]:
        early_stopping_counter -= 1 # update early stopping counter
    else:
        early_stopping_counter = early_stopping_patience # reset early stopping counter
  if early_stopping_counter == 0: # early stopping
      break

EPOCH 0


epoch 0 - train: 3it [00:08,  2.95s/it]
epoch 0 - validation: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


- saved best model: val_loss = 0.5548129081726074 val_accuracy = 0.7049179673194885
EPOCH 1


epoch 1 - train: 3it [00:02,  1.11it/s]
epoch 1 - validation: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


- saved best model: val_loss = 0.5287127494812012 val_accuracy = 0.737704873085022
EPOCH 2


...

epoch 197 - train: 3it [00:02,  1.09it/s]
epoch 197 - validation: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


EPOCH 198


epoch 198 - train: 3it [00:03,  1.05s/it]
epoch 198 - validation: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


EPOCH 199


epoch 199 - train: 3it [00:02,  1.11it/s]
epoch 199 - validation: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


In [86]:
from torch.utils.data import DataLoader

# Dataset with augmentation for visualization
test_dataset = VideoFrameDataset(root_path="data/split/FRAMES/TEST_SET/",
                            num_segments=3,
                            frames_per_segment=1,
                            ########transform=albumentations.Compose([preprocessing, augmentation], p=1.),
                            transform=albumentations.Compose([preprocessing], p=1.),
                            test_mode=True,
                            )

dataloader_params = {"batch_size": batch_size, "num_workers": 0, "pin_memory": True}
test_loader = DataLoader(test_dataset, shuffle=False, **dataloader_params)


############## FOR DEBUGGING PURPOSES ##############
# selected_model = project_root / "src" / "slow_r50_trained" / "best_model.pth"
# model = FireDetectionModelFactory.create_model(model_name, num_classes, to_train).to(device)
####################################################

model = X3D_xs(num_classes, to_train).to(device)
model.load_state_dict(torch.load(os.path.join(experiment_name,'best_model.pth')))

# model.load_state_dict(torch.load('../weights/X3Dxs.pth', map_location=torch.device('cpu')))

Y, Y_hat = [], []
with torch.no_grad():
  model.eval()
  for X, y in test_loader:
    Y.append(y)
    out = (output_activation(model(X.to(device))) > .5).squeeze().cpu()
    # ensure out has always one dimension
    out = out.view(-1)  # or out = out.reshape(-1)
    Y_hat.append(out)

Y = torch.cat(Y).float()
Y_hat = torch.cat(Y_hat).float()  # Concatenate along the batch dimension (dim=0)
accuracy = (Y == Y_hat).float().mean().item()
print("Accuracy on the test set: {:.4f}".format(accuracy))

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


Accuracy on the test set: 0.8923
